# Question
Chargement des fichiers pour les années 2021, 2022 

In [17]:
import requests
import os

# URLs des fichiers CSV pour les années 2021 et 2022
urls = {
    'usagers-2022.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094229/usagers-2022.csv",
    'vehicules-2022.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094147/vehicules-2022.csv",
    'lieux-2022.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094112/lieux-2022.csv",
    'caracteristiques-2022.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-093927/carcteristiques-2022.csv",
    'usagers-2021.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2022/20231009-140337/usagers-2021.csv",
    'vehicules-2021.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2020/20221024-113925/vehicules-2021.csv",
    'lieux-2021.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2020/20221024-113901/lieux-2021.csv",
    'caracteristiques-2021.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2020/20221024-113743/carcteristiques-2021.csv"
}

# Création du répertoire 'data' s'il n'existe pas
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

# Téléchargement des fichiers
for filename, url in urls.items():
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(data_dir, filename), 'wb') as f:
            f.write(response.content)
        print(f"Téléchargement réussi : {filename}")
    else:
        print(f"Erreur lors du téléchargement de {filename}")


Téléchargement réussi : usagers-2022.csv
Téléchargement réussi : vehicules-2022.csv
Téléchargement réussi : lieux-2022.csv
Téléchargement réussi : caracteristiques-2022.csv
Téléchargement réussi : usagers-2021.csv
Téléchargement réussi : vehicules-2021.csv
Téléchargement réussi : lieux-2021.csv
Téléchargement réussi : caracteristiques-2021.csv


# Question Supplémentaire
Netoyage du repertoire data après un eventuel telechargement des fichiers .csv

In [9]:
import os

# Chemin du répertoire 'data'
data_dir = 'data'

# Vérifier si le répertoire existe
if os.path.exists(data_dir):
    # Itérer sur tous les fichiers du répertoire
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)
        # Vérifier si c'est un fichier (et non un sous-répertoire)
        if os.path.isfile(file_path):
            os.remove(file_path)  # Supprimer le fichier
            print(f"Fichier {filename} supprimé.")
else:
    print(f"Le répertoire {data_dir} n'existe pas.")


Fichier caracteristiques-2021.csv supprimé.
Fichier usagers-2021.csv supprimé.
Fichier caracteristiques-2022.csv supprimé.
Fichier vehicules-2021.csv supprimé.
Fichier lieux-2022.csv supprimé.
Fichier usagers-2022.csv supprimé.
Fichier vehicules-2022.csv supprimé.
Fichier lieux-2021.csv supprimé.


# Question
Création de dataframes correspondants au fichiers des années 2021, 2022 et
nettoyage des données

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType
from pyspark.sql.functions import lpad, concat, lit, to_date
from pyspark.ml.feature import StringIndexer

# Initialiser SparkSession
spark = SparkSession.builder.appName("Spark session....").getOrCreate()
#spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

# Définir le schéma sans spécifier de types pour lat et long
schema_caracteristiques2021 = StructType([
    StructField("Num_Acc", LongType(), True),
    StructField("jour", IntegerType(), True),
    StructField("mois", IntegerType(), True),
    StructField("an", IntegerType(), True),
    StructField("hrmn", StringType(), True),
    StructField("lum", IntegerType(), True),
    StructField("dep", IntegerType(), True),
    StructField("com", IntegerType(), True),
    StructField("agg", IntegerType(), True),
    StructField("int", IntegerType(), True),
    StructField("atm", IntegerType(), True),
    StructField("col", IntegerType(), True),
    StructField("adr", StringType(), True),
    StructField("lat", StringType(), True),  # Lire comme StringType
    StructField("long", StringType(), True)  # Lire comme StringType
])

schema_caracteristiques2022 = StructType([
    StructField("Accident_Id", LongType(), True),
    StructField("jour", IntegerType(), True),
    StructField("mois", IntegerType(), True),
    StructField("an", IntegerType(), True),
    StructField("hrmn", StringType(), True),
    StructField("lum", IntegerType(), True),
    StructField("dep", IntegerType(), True),
    StructField("com", IntegerType(), True),
    StructField("agg", IntegerType(), True),
    StructField("int", IntegerType(), True),
    StructField("atm", IntegerType(), True),
    StructField("col", IntegerType(), True),
    StructField("adr", StringType(), True),
    StructField("lat", StringType(), True),  # Lire comme StringType
    StructField("long", StringType(), True)  # Lire comme StringType
])



schema_lieux = StructType([
    StructField("Num_Acc", LongType(), True),
    StructField("catr", IntegerType(), True),  
    StructField("voie", StringType(), True),
    StructField("V1", StringType(), True),
    StructField("V2", StringType(), True),
    StructField("circ", IntegerType(), True),
    StructField("nbv", IntegerType(), True),
    StructField("vosp", IntegerType(), True),
    StructField("prof", IntegerType(), True),
    StructField("pr", IntegerType(), True),
    StructField("pr1", IntegerType(), True),
    StructField("plan", IntegerType(), True),
    StructField("lartpc", IntegerType(), True),
    StructField("larrout", IntegerType(), True),
    StructField("surf", IntegerType(), True),
    StructField("infra", IntegerType(), True), 
    StructField("situ", IntegerType(), True),
    StructField("vma", IntegerType(), True)
])


schema_usagers = StructType([
    StructField("Num_Acc", LongType(), True),
    StructField("id_usager", StringType(), True),  
    StructField("id_vehicule", StringType(), True),  
    StructField("num_veh", StringType(), True),
    StructField("place", IntegerType(), True),
    StructField("catu", IntegerType(), True),
    StructField("grav", IntegerType(), True),
    StructField("sexe", IntegerType(), True),
    StructField("an_nais", IntegerType(), True),
    StructField("trajet", IntegerType(), True),
    StructField("secu1", IntegerType(), True),
    StructField("secu2", IntegerType(), True),
    StructField("secu3", IntegerType(), True),
    StructField("locp", IntegerType(), True),
    StructField("actp", IntegerType(), True), 
    StructField("etatp", IntegerType(), True)
])



schema_vehicules = StructType([
    StructField("Num_Acc", LongType(), True),
    StructField("id_vehicule", StringType(), True),
    StructField("num_veh", StringType(), True),
    StructField("senc", IntegerType(), True),
    StructField("catv", IntegerType(), True),
    StructField("obs", IntegerType(), True),
    StructField("obsm", IntegerType(), True),
    StructField("choc", IntegerType(), True),
    StructField("manv", IntegerType(), True),
    StructField("motor", IntegerType(), True),
    StructField("occutc", IntegerType(), True)
])





# Fonction pour charger et nettoyer les données
def charger_et_nettoyer(schemaDataFrame, fichierCsv):
    dataFrame = spark.read.csv(fichierCsv, header=True, sep=";", schema=schemaDataFrame)


    
    # Nettoyage de 'long' et 'lat'
    if "long" in dataFrame.columns:
        dataFrame = dataFrame.withColumn("long", regexp_replace("long", ",", ".").cast("double"))
    if "lat" in dataFrame.columns:
        dataFrame = dataFrame.withColumn("lat", regexp_replace("lat", ",", ".").cast("double"))
    
    categorial = []
    # Traitement pour schema_caracteristiques
    if schemaDataFrame == schema_caracteristiques2021 or schemaDataFrame == schema_caracteristiques2022:
        categorial = ["lum", "agg", "int", "atm", "col"]
        dataFrame = dataFrame.withColumn("jour", lpad(col("jour"), 2, "0"))
        dataFrame = dataFrame.withColumn("mois", lpad(col("mois"), 2, "0"))
        date_str = concat(lpad(col("jour"), 2, "0"), lit("/"), lpad(col("mois"), 2, "0"), lit("/"), col("an"))
        dataFrame = dataFrame.withColumn("date", to_date(date_str, "dd/MM/yyyy"))
    
    # Traitement pour schema_lieux
    elif schemaDataFrame == schema_lieux:
        categorial = ["catr", "circ", "vosp", "prof", "plan", "surf", "infra", "situ"]
    
    # Traitement pour schema_vehicules
    elif schemaDataFrame == schema_vehicules:
        categorial = ["senc", "catv", "obs", "obsm", "choc", "manv", "motor"]
    
    # Traitement pour schema_usagers
    elif schemaDataFrame == schema_usagers:
        categorial = ["catu", "grav", "sexe", "trajet", "secu1", "secu2", "secu3", "locp", "actp", "etatp"]
    
    # Application de StringIndexer avec option "keep"
    for column in categorial:
        label_stringIdx = StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep")
        model = label_stringIdx.fit(dataFrame)
        dataFrame = model.transform(dataFrame)
    
    return dataFrame


def affichageDataFram(dataFrame, fichierCsv) : 
    print("#############################DataFrame du fichier .csv : ", fichierCsv, "#############################")
    dataFrame.show()
    print(fichierCsv)
    dataFrame.printSchema()


csvPathCararteristique2021 = "data/caracteristiques-2021.csv"
csvPathLieux2021 = "data/lieux-2021.csv"
csvPathUsagers2021 = "data/usagers-2021.csv"
csvPathVehicules2021 = "data/vehicules-2021.csv"

csvPathCararteristique2022 = "data/caracteristiques-2022.csv"
csvPathLieux2022 = "data/lieux-2022.csv"
csvPathUsagers2022 = "data/usagers-2022.csv"
csvPathVehicules2022 = "data/vehicules-2022.csv"


#Année 2021
df_caracteristiques_2021 = charger_et_nettoyer(schema_caracteristiques2021, csvPathCararteristique2021)
df_lieux_2021 = charger_et_nettoyer(schema_lieux, csvPathLieux2021)
df_usagers_2021 = charger_et_nettoyer(schema_usagers, csvPathUsagers2021)
df_vehicules_2021 = charger_et_nettoyer(schema_vehicules, csvPathVehicules2021)

affichageDataFram(df_caracteristiques_2021, csvPathCararteristique2021)
affichageDataFram(df_lieux_2021, csvPathLieux2021)
affichageDataFram(df_usagers_2021, csvPathUsagers2021)
affichageDataFram(df_vehicules_2021, csvPathVehicules2021)

print("###########################################################################################################################################")


#Année 2022
df_caracteristiques_2022 = charger_et_nettoyer(schema_caracteristiques2022, csvPathCararteristique2022)
# Renommez le champ `Accident_Id` en `Num_Acc` pour le DataFrame de 2022
df_caracteristiques_2022 = df_caracteristiques_2022.withColumnRenamed("Accident_Id", "Num_Acc") # Transformé Accident_Id en Num_Acc pour facilité l'union future

df_lieux_2022 = charger_et_nettoyer(schema_lieux, csvPathLieux2022)
df_usagers_2022 = charger_et_nettoyer(schema_usagers, csvPathUsagers2022)
df_vehicules_2022 = charger_et_nettoyer(schema_vehicules, csvPathVehicules2022)

affichageDataFram(df_caracteristiques_2022, csvPathCararteristique2022)
affichageDataFram(df_lieux_2022, csvPathLieux2022)
affichageDataFram(df_usagers_2022, csvPathUsagers2022)
affichageDataFram(df_vehicules_2022, csvPathVehicules2022)

#############################DataFrame du fichier .csv :  data/caracteristiques-2021.csv #############################
+------------+----+----+----+-----+---+---+-----+---+---+---+---+--------------------+-------------+------------+----------+---------+---------+---------+---------+---------+
|     Num_Acc|jour|mois|  an| hrmn|lum|dep|  com|agg|int|atm|col|                 adr|          lat|        long|      date|lum_index|agg_index|int_index|atm_index|col_index|
+------------+----+----+----+-----+---+---+-----+---+---+---+---+--------------------+-------------+------------+----------+---------+---------+---------+---------+---------+
|202100000001|  30|  11|2021|07:32|  2| 30|30319|  1|  1|  1|  1|              CD 981|    44.038958|    4.348022|2021-11-30|      3.0|      1.0|      0.0|      0.0|      3.0|
|202100000002|  25|  09|2021|14:20|  1| 51|51544|  1|  3|  1|  3|Aire de repos cro...|    49.242129|    4.554546|2021-09-25|      0.0|      1.0|      2.0|      0.0|      0.0|
|20210

# Question
Réunion des années 2021, 20222

In [19]:
csvPathCararteristique2021 = "data/caracteristiques-2021.csv"
csvPathLieux2021 = "data/lieux-2021.csv"
csvPathUsagers2021 = "data/usagers-2021.csv"
csvPathVehicules2021 = "data/vehicules-2021.csv"

csvPathCararteristique2022 = "data/caracteristiques-2022.csv"
csvPathLieux2022 = "data/lieux-2022.csv"
csvPathUsagers2022 = "data/usagers-2022.csv"
csvPathVehicules2022 = "data/vehicules-2022.csv"

#Année 2021
df_caracteristiques_2021 = charger_et_nettoyer(schema_caracteristiques2021, csvPathCararteristique2021)
df_lieux_2021 = charger_et_nettoyer(schema_lieux, csvPathLieux2021)
df_usagers_2021 = charger_et_nettoyer(schema_usagers, csvPathUsagers2021)
df_vehicules_2021 = charger_et_nettoyer(schema_vehicules, csvPathVehicules2021)

#Année 2022
df_caracteristiques_2022 = charger_et_nettoyer(schema_caracteristiques2022, csvPathCararteristique2022)
df_lieux_2022 = charger_et_nettoyer(schema_lieux, csvPathLieux2022)
df_usagers_2022 = charger_et_nettoyer(schema_usagers, csvPathUsagers2022)
df_vehicules_2022 = charger_et_nettoyer(schema_vehicules, csvPathVehicules2022)



# Réaliser une union (réunion) des DataFrames de chaque catégorie pour les années 2021 et 2022
df_caracteristiques_union = df_caracteristiques_2021.union(df_caracteristiques_2022)
df_lieux_union = df_lieux_2021.union(df_lieux_2022)
df_usagers_union = df_usagers_2021.union(df_usagers_2022)
df_vehicules_union = df_vehicules_2021.union(df_vehicules_2022)

# Afficher les résultats pour vérifier
print("DataFrame Caractéristiques Union 2021 et 2022")
df_caracteristiques_union.show()

print("DataFrame Lieux Union 2021 et 2022")
df_lieux_union.show()

print("DataFrame Usagers Union 2021 et 2022")
df_usagers_union.show()

print("DataFrame Vehicules Union 2021 et 2022")
df_vehicules_union.show()

DataFrame Caractéristiques Union 2021 et 2022
+------------+----+----+----+-----+---+---+-----+---+---+---+---+--------------------+-------------+------------+----------+---------+---------+---------+---------+---------+
|     Num_Acc|jour|mois|  an| hrmn|lum|dep|  com|agg|int|atm|col|                 adr|          lat|        long|      date|lum_index|agg_index|int_index|atm_index|col_index|
+------------+----+----+----+-----+---+---+-----+---+---+---+---+--------------------+-------------+------------+----------+---------+---------+---------+---------+---------+
|202100000001|  30|  11|2021|07:32|  2| 30|30319|  1|  1|  1|  1|              CD 981|    44.038958|    4.348022|2021-11-30|      3.0|      1.0|      0.0|      0.0|      3.0|
|202100000002|  25|  09|2021|14:20|  1| 51|51544|  1|  3|  1|  3|Aire de repos cro...|    49.242129|    4.554546|2021-09-25|      0.0|      1.0|      2.0|      0.0|      0.0|
|202100000003|  15|  07|2021|07:55|  1| 85|85048|  2|  1|  7|  6|15 rue Françoi

# Question
Résumés numériques

In [ ]:
from pyspark.sql.functions import mean, stddev, skewness, kurtosis, col, median
import plotly.graph_objects as go

#Answer
def stats(df, dataframeName):
    numeric_cols = [c for c, t in df.dtypes if t.startswith('string') == False and t != "date"]
    stats_df = []
    
    print("*************************Résumés numériques : Données statistiques de la dataframe : ", dataframeName, "************************************")
    for col_name in numeric_cols:
        mean_val = df.select(mean(col_name)).collect()[0][0]
        stddev_val = df.select(stddev(col_name)).collect()[0][0]
        skewness_val = df.select(skewness(col_name)).collect()[0][0]
        kurtosis_val = df.select(kurtosis(col_name)).collect()[0][0]
        quantiles = df.stat.approxQuantile(col_name, [0.25, 0.5, 0.75], 0.05)
        
        iqr = quantiles[2] - quantiles[0]  # Interquartile range
        stats_df.append((col_name, mean_val, quantiles[1], quantiles[0], quantiles[2], iqr, stddev_val, skewness_val, kurtosis_val))
        
        # Plotly Boxplot
        fig = go.Figure()
        fig.add_trace(go.Box(y=df.select(col_name).rdd.flatMap(lambda x: x).collect(), name=col_name))
        fig.update_layout(title=f"Boxplot of {col_name}")
        fig.show()
        
    stats_df = spark.createDataFrame(stats_df, ["Column", "Mean", "Median", "Q1", "Q3", "IQR", "StdDev", "Skewness", "Kurtosis"])
    stats_df.show() 



#REMARQUE : LES AFFICHAGES ETANT ENERGIVORES, NOUS AVONS PRIVILEGIER DE METTRE DE NE PAR AFFICHER TOUS LES RESUMÉS, VOUS POUVEZ LES AFFICHER EN RETIRANT LES COMMANTAIRES POUR LES AUTRES APPEL À LA FONCTION STATS

#Données statistiques pour dataframe 2021
stats(df_caracteristiques_2021.na.drop(), csvPathCararteristique2021)
#stats(df_lieux_2021.na.drop(), csvPathLieux2021)
#stats(df_usagers_2021.na.drop(), csvPathUsagers2021)
#stats(df_vehicules_2021.na.drop(), csvPathVehicules2021)



#Résumé numerique pour les dataframes 2022
"""stats(df_caracteristiques_2022.na.drop()), csvPathCararteristique2022
stats(df_lieux_2022.na.drop(), csvPathLieux2022)
stats(df_usagers_2022.na.drop(), csvPathUsagers2022)
stats(df_vehicules_2022.na.drop(), csvPathVehicules2022)"""


#Résumé numerique pour les dataframes(Réunion)
"""stats(df_caracteristiques_union.na.drop(), "Union dataframe 2021 et 2022 de caracteristique")
stats(df_lieux_union.na.drop(), "Union dataframe 2021 et 2022 de lieux")
stats(df_usagers_union.na.drop(), "Union dataframe 2021 et 2022 de usagers")
stats(df_vehicules_union.na.drop(), "Union dataframe 2021 et 2022 de vehicules")"""

# Question
Visualisation
Construisez des objets graphiques propres à visualiser :
- Répartition des accidents sur la semaine (jours et heures)
- Répartition des accidents sur les mois de l’année

In [ ]:
#Answer 
import pandas as pd
import plotly.express as px

def pretraitement(dataFrame): 
    # Conversion en Pandas DataFrame pour la visualisation
    dataFrame = dataFrame.toPandas()

    # Convertir 'hrmn' en heure (en supposant que hrmn est au format HHMM ou HH:MM)
    dataFrame['hour'] = dataFrame['hrmn'].str[:2].astype(int)

    # Convertir 'jour' et 'mois' en type int (si ce n'est pas déjà fait)
    dataFrame['jour'] = dataFrame['jour'].astype(int)
    dataFrame['mois'] = dataFrame['mois'].astype(int)

    # Ajouter un champ 'weekday' pour le jour de la semaine
    # Assurez-vous que les colonnes 'an', 'mois', et 'jour' sont présentes et correctement nommées
    dataFrame['date'] = pd.to_datetime(dataFrame['an'].astype(str) + '-' + dataFrame['mois'].astype(str) + '-' + dataFrame['jour'].astype(str))
    dataFrame['weekday'] = dataFrame['date'].dt.dayofweek + 1
    
    #print(dataFrame.dtypes)
    return dataFrame  # Assurez-vous de retourner le DataFrame modifié si nécessaire



def repartition_accident_par_heure_sur_la_semaine(df) :
    
    dataFrame = pretraitement(df)     
    fig = px.histogram(dataFrame, x="hour", color="weekday",
                    category_orders={"weekday": [1, 2, 3, 4, 5, 6, 7]},
                    labels={"weekday": "Jour de la semaine", "hour": "Heure"},
                    title="Répartition des accidents par heure sur la semaine",
                    barmode='group')
    fig.show()
    
    
def repartition_accident_par_jour_sur_la_semaine(df) :    
    dataFrame = pretraitement(df)
    jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    dataFrame['weekday_name'] = dataFrame['weekday'].apply(lambda x: jours[x-1])

    fig = px.histogram(dataFrame, x="weekday_name",
                    category_orders={"weekday_name": jours},
                    labels={"weekday_name": "Jour de la semaine"},
                    title="Répartition des accidents par jour de la semaine",
                    barmode='group')
    fig.show()    
    

def repartition_accident_par_mois(df):
    dataFrame = pretraitement(df)
    fig = px.histogram(dataFrame, x="mois",
                    title="Répartition des accidents par mois",
                    labels={"mois": "Mois"},
                    category_orders={"mois": list(range(1, 13))})  # Convertissez range en list ici
    fig.update_xaxes(type='category', 
                    tickmode='array', 
                    tickvals=list(range(1, 13)),  # Conversion de range en list ici également
                    ticktext=['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun', 'Juil', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc'])
    fig.show()


"""print("#########################################Repartition des données accidents pour 2021#############################################################")
repartition_accident_par_heure_sur_la_semaine(df_caracteristiques_2021)
repartition_accident_par_jour_sur_la_semaine(df_caracteristiques_2021)
repartition_accident_par_mois(df_caracteristiques_2021)"""

print("#########################################Repartition des données accidents pour 2022#############################################################")
repartition_accident_par_heure_sur_la_semaine(df_caracteristiques_2022)
repartition_accident_par_jour_sur_la_semaine(df_caracteristiques_2022)
repartition_accident_par_mois(df_caracteristiques_2022)

# Question
Profil des usagers

In [ ]:
#Answer 
from pyspark.sql.functions import when
import plotly.express as px

def pretraitementProfileUsagers(dfUsagers, dfLieux) : 
    
    # Joindre les DataFrames df_usagers et df_lieux sur 'Num_Acc'
    df_joint = dfUsagers.join(dfLieux, "Num_Acc")

    # Catégoriser la catégorie de route, la gravité des blessures, et le sexe des usagers
    df_joint = df_joint.withColumn("Type_de_Route",
                when(col("catr") == 1, "Autoroute")
                .when(col("catr") == 2, "Route Nationale")
                .when(col("catr") == 3, "Route Départementale")
                .when(col("catr") == 4, "Voie Communale")
                .when(col("catr") == 5, "Hors réseau public")
                .when(col("catr") == 6, "Parc de stationnement")
                .when(col("catr") == 7, "Autre")
                .otherwise("Non spécifié"))

    df_joint = df_joint.withColumn("Gravite_Blessure",
                when(col("grav") == 1, "Indemne")
                .when(col("grav") == 2, "Tué")
                .when(col("grav") == 3, "Blessé hospitalisé")
                .when(col("grav") == 4, "Blessé léger")
                .otherwise("Non spécifié"))

    df_joint = df_joint.withColumn("Sexe",
                when(col("sexe") == 1, "Masculin")
                .when(col("sexe") == 2, "Féminin")
                .otherwise("Non spécifié"))

    # Convertir en Pandas DataFrame pour la visualisation
    df_joint_pd = df_joint.toPandas()
    
    #print(df_joint_pd.dtypes)
    return df_joint_pd




# Répartition des usagers par Type de Route et Gravité des blessures
def rapartitionRouteEtGravite(df_joint_pd) : 
    # Répartition des usagers par Type de Route et Gravité des blessures
    fig = px.histogram(df_joint_pd, x="Type_de_Route", color="Gravite_Blessure",
            title="Répartition des usagers par type de route et gravité des blessures",
            labels={"Gravite_Blessure": "Gravité des blessures", "Type_de_Route": "Type de Route"},
            barmode='group',
            category_orders={"Gravite_Blessure": ["Indemne", "Tué", "Blessé hospitalisé", "Blessé léger"]})
    fig.show()


# Répartition des usagers par Sexe et Type de Route
def repartitionSexeEtTypeRoute(df_joint_pd) : 
    # Répartition des usagers par Sexe et Type de Route
    fig = px.histogram(df_joint_pd, x="Sexe", color="Type_de_Route",
                title="Répartition des usagers par sexe et type de route",
                labels={"Sexe": "Sexe", "Type_de_Route": "Type de Route"},
                barmode='group')
    fig.show()
    

# Répartition des usagers par Sexe, Type de Route et Gravité des blessures
def repartitionSexeTypeRouteEtGravite(df_joint_pd) : 
    fig = px.histogram(df_joint_pd, x="Type_de_Route", color="Gravite_Blessure",
            facet_row="Sexe",
            title="Répartition des usagers par sexe, type de route et gravité des blessures",
            labels={"Gravite_Blessure": "Gravité des blessures", "Type_de_Route": "Type de Route", "Sexe": "Sexe"},
            barmode='group',
            category_orders={"Gravite_Blessure": ["Indemne", "Tué", "Blessé hospitalisé", "Blessé léger"], "Sexe": ["Masculin", "Féminin"]})
    fig.show()



print("###################################################Profil des usagers de 2021##################################################################")
rapartitionRouteEtGravite(pretraitementProfileUsagers(df_usagers_2021, df_lieux_2021))
repartitionSexeEtTypeRoute(pretraitementProfileUsagers(df_usagers_2021, df_lieux_2021))
repartitionSexeTypeRouteEtGravite(pretraitementProfileUsagers(df_usagers_2021, df_lieux_2021))


print("###################################################Profil des usagers de 2022##################################################################")



print("###################################################Profil des usagers de l'union 2021 et 2022##################################################################")

# Question
Accidents impliquant des cyclistes et/ou des piétons

In [49]:
#Answer
from pyspark.sql.functions import col, split, date_format, to_date, month, hour, to_timestamp, dayofweek

def avoirUsagersAccidentPieton(dfUsagers, dfCaracteristiques) : 
    if "Accident_Id" in dfCaracteristiques.columns:
        # Renommer la colonne `Accident_Id` en `Num_Acc` dans dfCaracteristiques pour correspondre à dfUsagers
        dfCaracteristiques = dfCaracteristiques.withColumnRenamed("Accident_Id", "Num_Acc")
    
    # Filtrer pour inclure les accidents impliquant des piétons ou des usagers potentiels cyclistes (à l'exclusion des codes 1 et 2)
    #df_pietons_et_cyclistes_potentiels = dfUsagers.filter((col("catu") == 3) | (~col("catu").isin([1, 2])))
    df_pietons_et_cyclistes_potentiels = dfUsagers.filter((col("catu") == 3))

    # Joindre df_pietons avec df_caracteristiques pour obtenir des détails complets sur les accidents impliquant des piétons
    df_accidents_pietons = df_pietons_et_cyclistes_potentiels.join(dfCaracteristiques, "Num_Acc")

    return df_accidents_pietons


def pretraitementAcidentsCyclistes(dfUsagers, dfCaracteristiques) : 
    df_accidents_pietons = avoirUsagersAccidentPieton(dfUsagers, dfCaracteristiques)


    # Prétraitement pour extraire heure, jour de la semaine et mois à partir de 'hrmn' et 'date'
    df_accidents_pietons = df_accidents_pietons \
            .withColumn("heure", split(col("hrmn"), ":").getItem(0).cast("integer")) \
            .withColumn("jour_semaine", dayofweek(col("date"))) \
            .withColumn("mois", month(col("date")))
        
    dfPandas = df_accidents_pietons.toPandas()
    return dfPandas


def affichageDataFramePandas(dataFrame, annee) : 
    print("############################Affichage de df pour pieton pour année ", annee, "###############################################")
    #print(dataFrame.dtypes)
    print(dataFrame)


def repartitionAccidentPietonHeursJourSemaine(dataFrame) : 
    fig = px.histogram(dataFrame, x="heure", color="jour_semaine",
            title="Répartition des accidents impliquant des piétons par heure de la semaine",
            labels={"heure": "Heure", "jour_semaine": "Jour de la semaine"},
            barmode='group', 
            category_orders={"jour_semaine": [1, 2, 3, 4, 5, 6, 7]})
    fig.show()



def repartitionAccidentPietonJourSemaine(dataFrame) : 
    fig = px.histogram(dataFrame, x="jour_semaine",
            title="Répartition des accidents impliquant des piétons par jour de la semaine",
            labels={"jour_semaine": "Jour de la semaine"},
            category_orders={"jour_semaine": [1, 2, 3, 4, 5, 6, 7]})
    fig.update_xaxes(tickvals=list(range(1, 8)), ticktext=['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche'])
    fig.show()


def repartitionAccidentPietonMoisAnnee(dataFrame) : 
    fig = px.histogram(dataFrame, x="mois",
            title="Répartition des accidents impliquant des piétons par mois de l'année",
            labels={"mois": "Mois"},
            category_orders={"mois": list(range(1, 13))})
    fig.update_xaxes(tickvals=list(range(1, 13)), ticktext=['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Juin', 'Juil', 'Août', 'Sept', 'Oct', 'Nov', 'Déc'])
    fig.show()



dfAccientPieton2021 = pretraitementAcidentsCyclistes(df_usagers_2021, df_caracteristiques_2021)
affichageDataFramePandas(dfAccientPieton2021, "2021")
repartitionAccidentPietonHeursJourSemaine(dfAccientPieton2021)
repartitionAccidentPietonJourSemaine(dfAccientPieton2021)
repartitionAccidentPietonMoisAnnee(dfAccientPieton2021)


"""
dfAccientPieton2022 = pretraitementAcidentsCyclistes(df_usagers_2022, df_caracteristiques_2022)
affichageDataFramePandas(dfAccientPieton2022, "2022")
repartitionAccidentPietonHeursJourSemaine(dfAccientPieton2022)
repartitionAccidentPietonJourSemaine(dfAccientPieton2022)
repartitionAccidentPietonMoisAnnee(dfAccientPieton2022)"""


############################Affichage de df pour pieton pour année  2021 ###############################################
           Num_Acc id_usager id_vehicule num_veh  place  catu  grav  sexe  \
0     202100000003   267 635     201 761     A01     10     3     3     2   
1     202100000023   267 576     201 719     A01     10     3     4     1   
2     202100000029   267 559     201 708     A01     10     3     3     1   
3     202100000030   267 557     201 707     A01     10     3     3     2   
4     202100000033   267 548     201 702     A01     10     3     3     2   
...            ...       ...         ...     ...    ...   ...   ...   ...   
9512  202100056448   133 998     101 002     A01     10     3     4     1   
9513  202100056460   133 971     100 982     A01     10     3     2     2   
9514  202100056466   133 947     100 970     A01     10     3     3     1   
9515  202100056476   133 918     100 951     A01     10     3     3     2   
9516  202100056487   133 878    

'\ndfAccientPieton2022 = pretraitementAcidentsCyclistes(df_usagers_2022, df_caracteristiques_2022)\naffichageDataFramePandas(dfAccientPieton2022, "2022")\nrepartitionAccidentPietonHeursJourSemaine(dfAccientPieton2022)\nrepartitionAccidentPietonJourSemaine(dfAccientPieton2022)\nrepartitionAccidentPietonMoisAnnee(dfAccientPieton2022)'

# Construisez des objets graphiques propres à visualiser les caractéristiques des lieux où se sont produits ces accidents.


In [59]:
def  repartitionDuCaracteristiqueEclairage(df_visualisation) : 
    lum_labels = {
        1: 'Plein jour',
        2: 'Crépuscule ou aube',
        3: 'Nuit sans éclairage public',
        4: 'Nuit avec éclairage public non allumé',
        5: 'Nuit avec éclairage public allumé'
    }
    df_visualisation['lum_label'] = df_visualisation['lum'].map(lum_labels)
    fig_lum = px.histogram(df_visualisation, x='lum_label', title='Conditions d’éclairage lors des accidents')
    fig_lum.show()


def repartitionDuCaracteristiqueLocationsation(df_visualisation) :
    agg_labels = {1: 'Hors agglomération', 2: 'En agglomération'}
    df_visualisation['agg_label'] = df_visualisation['agg'].map(agg_labels)
    fig_agg = px.histogram(df_visualisation, x='agg_label', title='Localisation des accidents')
    fig_agg.show()


def repartitionDuCaracteristiqueAthmospherique(df_visualisation) : 
    atm_labels = {
        1: 'Normale',
        2: 'Pluie légère',
        3: 'Pluie forte',
        4: 'Neige - grêle',
        5: 'Brouillard - fumée',
        6: 'Vent fort - tempête',
        7: 'Temps éblouissant',
        8: 'Temps couvert',
        9: 'Autre'
    }
    df_visualisation['atm_label'] = df_visualisation['atm'].map(atm_labels)
    fig_atm = px.histogram(df_visualisation, x='atm_label', title='Conditions atmosphériques lors des accidents')
    fig_atm.show()

def repartitionDuCaracteristiqueCollision(df_visualisation) : 
    col_labels = {
        1: 'Deux véhicules - frontale',
        2: 'Deux véhicules – par l’arrière',
        3: 'Deux véhicules – par le coté',
        4: 'Trois véhicules et plus – en chaîne',
        5: 'Trois véhicules et plus - collisions multiples',
        6: 'Autre collision',
        7: 'Sans collision'
    }
    df_visualisation['col_label'] = df_visualisation['col'].map(col_labels)
    fig_col = px.histogram(df_visualisation, x='col_label', title='Type de collision lors des accidents')
    fig_col.show()



def repartitionDuCaracteristiqueIntersection(df_visualisation):
    int_labels = {
        1: 'Hors intersection',
        2: 'Intersection en X',
        3: 'Intersection en T',
        4: 'Intersection en Y',
        5: 'Intersection à plus de 4 branches',
        6: 'Giratoire',
        7: 'Place',
        8: 'Passage à niveau',
        9: 'Autre intersection'
    }
    # Correction: Appliquer le mappage à la colonne 'int' pour obtenir les étiquettes des intersections
    df_visualisation['int_label'] = df_visualisation['int'].map(int_labels)
    fig_int = px.histogram(df_visualisation, x='int_label', title='Répartition des types d\'intersections')
    fig_int.show()


# Recuperer les usagers pietons qui contient aussi le contenue le caracteristiques des lieux des accident
dfCaracteristiquesDesLieuxPourPietons2021 = avoirUsagersAccidentPieton(df_usagers_2021, df_caracteristiques_2021).toPandas()
repartitionDuCaracteristiqueEclairage(dfCaracteristiquesDesLieuxPourPietons2021)
repartitionDuCaracteristiqueLocationsation(dfCaracteristiquesDesLieuxPourPietons2021)
repartitionDuCaracteristiqueAthmospherique(dfCaracteristiquesDesLieuxPourPietons2021)
repartitionDuCaracteristiqueCollision(dfCaracteristiquesDesLieuxPourPietons2021)
repartitionDuCaracteristiqueIntersection(dfCaracteristiquesDesLieuxPourPietons2021)

# Question
Usage des types composites

In [ ]:
#Answer 


# Question
Sauvegarde au format parquet

In [ ]:
#Answer 
